#### GROUP 17: Shuvojyoti Singha, Artjom Smorgulenko, Kaan Özkiliç, Samuel Pasierb

# Stroke Prediction using Support Vector Machine (SVM)

## Background
This notebook uses SVM algorithm to evaluate a dataset where stroke prediction data is recorded. The dataset includes patient information like gender, glucose level and other health related data; and the aim is to predict if patient is likely to have a stroke based on trained data.

This notebook file evaluates how different parameter values and kernels affect the accuracy of SVM.

# Constants
Defined constant variables for readibility and to avoid repetition.

In [12]:
ID = "ID"
GENDER = "Gender"
AGE = "Age"
HYPERTENSION = "Hypertension"
HEART_DISEASE = "Heart Disease"
EVER_MARRIED = "Ever Married"
WORK_TYPE = "Work Type"
RESIDENCE_TYPE = "Residence Type"
AVG_GLUCOSE_LEVEL = "Average Glucose Level"
BMI = "BMI"
SMOKING_STATUS = "Smoking Status"
STROKE = "Stroke"

# Import

In this notebook we've used certain libraries:
- **pandas**: for data handling,
- **numpy**: also for data handling,
- **matplotlib**: for plotting graphs,
- **sklearn**: for implementing SVM algorithm, model evaluation and training-testing split.

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import KFold   
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from matplotlib import pyplot as plt

# Loading Dataset

The dataset is downloaded from Kaggle. The dataset includes bunch of columns that are significant to predict stroke prediction.

*Link to the [the dataset from Kaggle](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset).*

In [14]:
df = pd.read_csv('healthcare-dataset-stroke-data.csv')
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


# Data Cleaning

The missing data is handled and categorical values are converted to numerical values.

- The missing values are filled with 0.
- ID and work type columns are dropped due to inconvenience.
- Categorical values are replaced with integers.

In [15]:
df.fillna(0, inplace=True)
df.isnull().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [16]:
if "id" in df.columns: df = df.drop(["id"], axis=1)
if "work_type" in df.columns: df = df.drop(["work_type"], axis=1)
print(df.columns)
df.columns = [GENDER, AGE, HYPERTENSION, HEART_DISEASE, EVER_MARRIED, RESIDENCE_TYPE, AVG_GLUCOSE_LEVEL, BMI, SMOKING_STATUS, STROKE]
df.head()

Index(['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status',
       'stroke'],
      dtype='object')


,Gender,Age,Hypertension,Heart Disease,Ever Married,Residence Type,Average Glucose Level,BMI,Smoking Status,Stroke
0,Male,67.0,0,1,Yes,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Rural,202.21,0.0,never smoked,1
2,Male,80.0,0,1,Yes,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Rural,174.12,24.0,never smoked,1


In [17]:
# gender: male = 0, female = 1, other = 2 ;
# ever married: yes = 1, no = 0 ;
# work type: children = 0, govt job = 1, never worked = 2, private = 3, self-employed = 4 ;
# residence type: rural = 0, urban = 1 ;
# smoking status: formerly smoked = 0, never smoked = 1, smokes = 2, unknown = 3

df[GENDER] = df[GENDER].replace({"Male": 0, "Female": 1, "Other": 2}).astype(int)
df[EVER_MARRIED] = df[EVER_MARRIED].replace({"Yes": 1, "No": 0}).astype(int)
df[RESIDENCE_TYPE] = df[RESIDENCE_TYPE].replace({"Rural": 0, "Urban": 1}).astype(int)
df[SMOKING_STATUS] = df[SMOKING_STATUS].replace({"formerly smoked": 0, "never smoked": 1, "smokes": 2, "Unknown": 3}).astype(int)
df.head()

C:\Users\artjo\AppData\Local\Temp\ipykernel_23380\4277707953.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[GENDER] = df[GENDER].replace({"Male": 0, "Female": 1, "Other": 2}).astype(int)
C:\Users\artjo\AppData\Local\Temp\ipykernel_23380\4277707953.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[EVER_MARRIED] = df[EVER_MARRIED].replace({"Yes": 1, "No": 0}).astype(int)
C:\Users\artjo\AppData\Local\Temp\ipykernel_23380\4277707953.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed i

,Gender,Age,Hypertension,Heart Disease,Ever Married,Residence Type,Average Glucose Level,BMI,Smoking Status,Stroke
0,0,67.0,0,1,1,1,228.69,36.6,0,1
1,1,61.0,0,0,1,0,202.21,0.0,1,1
2,0,80.0,0,1,1,0,105.92,32.5,1,1
3,1,49.0,0,0,1,1,171.23,34.4,2,1
4,1,79.0,1,0,1,0,174.12,24.0,1,1


# Separating Data and Target

The data is seperated from target values. The `df_data` has input values and `df_target` has the results if stroke occurred.

In [18]:
df_data = df.iloc[:, :-1]
df_target = df.iloc[:, -1]
print(df_data)
print(df_target)

      Gender   Age  Hypertension  Heart Disease  Ever Married  Residence Type  \
0          0  67.0             0              1             1               1   
1          1  61.0             0              0             1               0   
2          0  80.0             0              1             1               0   
3          1  49.0             0              0             1               1   
4          1  79.0             1              0             1               0   
...      ...   ...           ...            ...           ...             ...   
5105       1  80.0             1              0             1               1   
5106       1  81.0             0              0             1               1   
5107       1  35.0             0              0             1               0   
5108       0  51.0             0              0             1               0   
5109       1  44.0             0              0             1               1   

      Average Glucose Level

# Splitting into Training and Testing Sets

The dataset is splitted into training and testing sets. In this cell, 80/20 split is used.

In [19]:
data_train, data_test, target_train, target_test = train_test_split(df_data, df_target, test_size=0.2, shuffle=True, random_state=0)

# Use case identification

SVM’s high-dimensional classification ability is beneficial in face detection and in handwritten text recognition.

<b>Face detection:</b> 
<br>&emsp;SVM can clasifiy wheter an image contains a face or no face which can help in face tracking in videography or security fields and it is used as a basis for face analysis and face recognition<br>
<b>Handwritten text recognition:</b> 
<br>&emsp;SVM can recognize handwritten character and correctly classify them which helps create a digital version of a handwritten text for further processing in post services and signature recognition

# Algorithm Implementation

## Linear

In [ ]:
svm_clf = SVC(kernel='linear', C=1)
svm_clf.fit(data_train, target_train)

target_pred = svm_clf.predict(data_test)

print("Accuracy:", accuracy_score(target_test, target_pred))
print("Classification report:\n", classification_report(target_test, target_pred))
print("Confusion matrix:\n", confusion_matrix(target_test, target_pred))

Accuracy: 0.9471624266144814
Classification report:
               precision    recall  f1-score   support

           0       0.95      1.00      0.97       968
           1       0.00      0.00      0.00        54

    accuracy                           0.95      1022
   macro avg       0.47      0.50      0.49      1022
weighted avg       0.90      0.95      0.92      1022

Confusion matrix:
 [[968   0]
 [ 54   0]]


c:\Users\artjo\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\artjo\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\artjo\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Polynomial

In [26]:
polynomial_svm_clf = make_pipeline(
    PolynomialFeatures(degree=3),
    StandardScaler(),
    LinearSVC(C=10, max_iter=10000, dual=True, random_state=42)
    )
polynomial_svm_clf.fit(df_data, df_target)

c:\Users\artjo\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('standardscaler', StandardScaler()),
                ('linearsvc',
                 LinearSVC(C=10, dual=True, max_iter=10000, random_state=42))])

# Parameter Experimentation

In [28]:
svm_clf2 = SVC(kernel='linear', C=5)
svm_clf2.fit(data_train, target_train)

target_pred = svm_clf2.predict(data_test)

print("Accuracy:", accuracy_score(target_test, target_pred))
print("Classification report:\n", classification_report(target_test, target_pred))
print("Confusion matrix:\n", confusion_matrix(target_test, target_pred))

Accuracy: 0.9471624266144814
Classification report:
               precision    recall  f1-score   support

           0       0.95      1.00      0.97       968
           1       0.00      0.00      0.00        54

    accuracy                           0.95      1022
   macro avg       0.47      0.50      0.49      1022
weighted avg       0.90      0.95      0.92      1022

Confusion matrix:
 [[968   0]
 [ 54   0]]


c:\Users\artjo\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\artjo\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\artjo\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
